# Find out how the bird population in NC has changed over time #


In [68]:
#imports
import pandas as pd
import numpy as np

In [45]:
bird_names_file = 'birbs.csv' #file path

nc_birbs = pd.read_csv(bird_names_file)
nc_birbs.head()

,Species List,1966,1967,1968,1969,1970,1971,1972,1973,1974,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,Route Count,14,13,21,20,17,22,22,22,21,...,61,75,83,82,81,75,79,77,82,80
1,Canada Goose,0,0,0,0,0,0,0,0,0,...,144,496,365,350,211,423,352,289,320,325
2,Wood Duck,3,0,1,2,5,2,5,16,1,...,19,35,23,33,52,48,57,71,69,79
3,Mallard,0,0,0,0,0,0,0,0,0,...,32,33,18,19,11,57,44,20,59,35
4,American Black Duck,0,0,0,0,0,0,1,0,20,...,0,0,0,0,0,0,0,0,2,0


## Find the net flux of birds through time##
### Story Point 4 ###
#### Species lost and gained through time ####

In [46]:
birds = nc_birbs.copy()

In [47]:
birds.index = birds['Species List']
birds.drop('Species List', axis=1, inplace=True)
birds.drop(['Route Count','Total Species','Total individuals'], inplace=True)
birds.head()

,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
Species List,,,,,,,,,,,,,,,,,,,,,
Canada Goose,0,0,0,0,0,0,0,0,0,0,...,144,496,365,350,211,423,352,289,320,325
Wood Duck,3,0,1,2,5,2,5,16,1,4,...,19,35,23,33,52,48,57,71,69,79
Mallard,0,0,0,0,0,0,0,0,0,0,...,32,33,18,19,11,57,44,20,59,35
American Black Duck,0,0,0,0,0,0,1,0,20,2,...,0,0,0,0,0,0,0,0,2,0
Northern Bobwhite,848,717,919,812,693,1028,1024,925,888,811,...,394,542,519,488,420,322,331,283,310,302


In [48]:
def start_year(bird, years, name):
    """This function takes a single bird's data and returns the name of the bird and the first year it was
    seen in the data"""
    for yr in range(len(bird)):
        year = years[yr]
        if bird[yr] != 0:
            break
    return [name, year]

In [49]:
def last_year(bird,years,name):
    """This function takes a single bird's data and returns the name of the bird and the first year it was
    missing from the data"""
    for yr in reversed(range(len(bird))):
        year = years[yr]
        if bird[yr] != 0:
            break
    return [name, int(year)+1]

In [50]:
def last_list(df):
    """Takes the birds dataframe and for each species finds when it was last seen in the bird count"""
    end_df = pd.DataFrame(columns=['Species', 'Year'])
    birds = df.index
    years = df.columns
    for bird in birds:
        row = pd.DataFrame(last_year(df.loc[bird],df.columns,bird)).transpose()
        row.columns = ['Species','Year']
        end_df = pd.concat([end_df,row])
    lost = end_df[end_df.Year != 2019] #only return those species that have dissapeared
    return lost


In [51]:
def start_list(df):
    """Takes the birds dataframe and for each species finds when it was first seen in the bird count"""
    start_df = pd.DataFrame(columns=['Species', 'Year'])
    birds = df.index
    years = df.columns
    for bird in birds:
        row = pd.DataFrame(start_year(df.loc[bird],df.columns,bird)).transpose()
        row.columns = ['Species','Year']
        start_df = pd.concat([start_df,row])
    introduced = start_df[start_df.Year != '1966'] #do not include those that were present at the start of the data
    return introduced
        

In [52]:
species_introduced = start_list(birds).reset_index(drop=True)
species_lost = last_list(birds).reset_index(drop=True)
species_lost.head()

,Species,Year
0,American Black Duck,2018
1,Ruffed Grouse,2017
2,Pied-billed Grebe,2005
3,Common Ground-Dove,1970
4,Black-billed Cuckoo,2016


##### output for tableau #####

In [53]:
species_introduced.to_csv('introduced.csv')
species_lost.to_csv('lost.csv')

#### New and lost species count through time and net flux ####

In [54]:
count_intro = species_introduced.groupby('Year').count()
count_lost = species_lost.groupby('Year').count()

years = birds.columns #list all years for for loop

#add all years to the dataframe that didn't lose or gain species
for yr in years:
    if yr not in count_intro.index:
        count_intro = pd.concat([count_intro, pd.DataFrame({'Species': 0}, index = [yr])])
for yr in years.astype(int):
    if yr not in count_lost.index:
        count_lost = pd.concat([count_lost, pd.DataFrame({'Species': 0}, index = [yr])])

count_intro.columns = ['Introduced']
count_lost.columns = ['Lost']
count_intro.index = count_intro.index.astype(int)

flux = count_intro.merge(count_lost, left_index=True, right_index=True).sort_index()
flux['Net Change'] = flux.Introduced - flux.Lost
flux.head()

,Introduced,Lost,Net Change
1966,0,0,0
1967,9,0,9
1968,22,0,22
1969,6,0,6
1970,4,1,3


##### output for tableau #####

In [55]:
flux.to_csv('bird_flux.csv')

## Create Tidy Data for Tableau ##
### Story Point 1 and 2 ###

In [56]:
breeding = nc_birbs.transpose()
breeding.reset_index(inplace=True)
breeding.iloc[0,0] = 'Year'
breeding.columns = breeding.iloc[0]
breeding = breeding.iloc[1:]
breeding.drop('Route Count', axis=1, inplace=True)
breeding.head()

,Year,Canada Goose,Wood Duck,Mallard,American Black Duck,Northern Bobwhite,Ruffed Grouse,Wild Turkey,Pied-billed Grebe,Rock Pigeon,...,Summer Tanager,Scarlet Tanager,Northern Cardinal,Rose-breasted Grosbeak,Blue Grosbeak,Indigo Bunting,Painted Bunting,Dickcissel,Total Species,Total individuals
1,1966,0,3,0,0,848,0,0,0,12,...,61,3,603,0,81,270,6,0,100,10839
2,1967,0,0,0,0,717,0,0,0,38,...,46,0,694,0,84,347,9,0,99,10565
3,1968,0,1,0,0,919,0,1,0,60,...,60,21,915,7,108,476,3,0,125,14626
4,1969,0,2,0,0,812,0,2,0,111,...,40,25,884,0,113,499,3,0,121,13536
5,1970,0,5,0,0,693,0,0,0,111,...,40,23,692,0,80,326,4,0,114,12399


In [65]:
#melt bird columns to make each row an observation of a single species count in a single year
output = pd.melt(breeding, id_vars=['Year', 'Total Species','Total individuals'])

##### output for tableau #####

In [50]:
output.to_csv('nc_breeding_birds_66_18.csv')

## Find the average top 10 species for each group of years

In [66]:
#make year datetime index and turn numeric columns into int type
output.Year = pd.to_datetime(output.Year)
output.index = output.Year
output.drop('Year', axis=1, inplace=True)
output.columns = ['tot_spec','tot_inds','species','birdcount']
cols = output.columns.drop('species')
output[cols] = output[cols].apply(pd.to_numeric,errors='coerce')
output.head()

,tot_spec,tot_inds,species,birdcount
Year,,,,
1966-01-01,100,10839,Canada Goose,0
1967-01-01,99,10565,Canada Goose,0
1968-01-01,125,14626,Canada Goose,0
1969-01-01,121,13536,Canada Goose,0
1970-01-01,114,12399,Canada Goose,0


In [128]:
#empty dataframe to input the ranks through the years
master = pd.DataFrame(columns=['species','tot_spec','tot_inds','birdcount','rank'])
master

,species,tot_spec,tot_inds,birdcount,rank


In [129]:
def avg_yrs(df, start, end, top):
    """This function takes the output df over a range of years and finds the average most populous species the top
    numeric value for 'top' is included"""
    dfyrs = df.loc[str(start):str(end),:]
    yrdf = dfyrs.groupby('species').mean().sort_values('birdcount', ascending=False).reset_index().iloc[:top,:]
    yrdf['rank'] = range(1,top+1)
    yrdf['years'] = start
    return yrdf

In [130]:
#Year groups defined by 10 equally spaced intervals
start = np.linspace(1966,2019, 10)
srt = [int(yr) for yr in start][:-1] #start leaves out end point
end = [int(yr)-1 for yr in start[1:]] #end leaves out start point

In [133]:
for s,e in zip(srt,end):
    df = avg_yrs(output,s,e,5)
    master = pd.concat([master,df],sort=False)
master

,birdcount,rank,species,tot_inds,tot_spec,years
0,1155.000000,1,House Sparrow,12393.000000,111.800000,1966.0
1,1106.600000,2,Common Grackle,12393.000000,111.800000,1966.0
2,804.800000,3,European Starling,12393.000000,111.800000,1966.0
3,797.800000,4,Northern Bobwhite,12393.000000,111.800000,1966.0
4,771.600000,5,Mourning Dove,12393.000000,111.800000,1966.0
...,...,...,...,...,...,...
0,3357.500000,1,American Crow,55088.166667,157.833333,2013.0
1,3230.833333,2,Mourning Dove,55088.166667,157.833333,2013.0
2,2858.666667,3,Northern Cardinal,55088.166667,157.833333,2013.0
3,2418.833333,4,European Starling,55088.166667,157.833333,2013.0


##### Save to Output #####

In [67]:
master.to_csv('grouped.csv')